IBM Capstone Project
The Battle of Neighborhoods - Final Report (Week 2)

Upload Libraries Required

In [1]:
import numpy as np
import time
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import requests
from pandas.io.json import json_normalize


!pip install geopy
from geopy.geocoders import Nominatim

!pip install folium
import folium
import folium
from folium import plugins


import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns


from sklearn.cluster import KMeans



print('Libraries imported.')

Libraries imported.


  Scenario and Background
I have been offered a great opportunity to work in Manhattan, NY. Although, I am very excited about it, I am a bit stress toward the process to secure a comparable place to live in Manhattan. Therefore, I decided to apply the learned skills during the Coursera course to explore ways to make sure my decision is factual and rewarding. Of course, there are alternatives to achieve the answer using available Google and Social media tools, but it rewarding doing it myself with learned tools.

Problem to be resolved:
The challenge to resolve is being able to find a rental apartment unit in Manhattan NY that offers similar characteristics and benefits to my current situation. Therefore, in order to set a basis for comparison, I want to find a renta unit subject to the following conditions:

Apartment with min 2 bedrooms with monthly rent not to exceed US$7000/month
Unit located within walking distance (<=1.0 mile, 1.6 km) from a subway metro station in Manhattan
Area with ammenities and venues similar to the ones described for current location ( See item 2.1)

FourSquare data and mapping techniques combined with data analysis will help resolve the key questions arisen. Lastly, this project is a good practical case toward the development of Data Science skills.

Description of the data and its sources that will be used to solve the problem

In order to make a good choice of a similar apartment in Manhattan NY, the following data is required: List/Information on neighborhoods form Manhattan with their Geodata ( latitud and longitud. List/Information about the subway metro stations in Manhattan with geodata. Listed apartments for rent in Manhattan area with descriptions ( how many beds, price, location, address) Venues and ammenities in the Manhattan neighborhoods (e.g. top 10) 2.3 sources and manipulation The list of Manhattan neighborhoods is worked out during LAb exercise during the course. A csv file was created which will be read in order to create a dataframe and its mapping. The csv file 'mh_neigh_data.csv' has the following below data structure. The file will be directly read to the Jupiter Notebook for convenience and space savings. The clustering of neighborhoods and mapping will be shown however. An algorythm was used to determine the geodata from Nominatim . The actual algorythm coding may be shown in 'markdown' mode becasues it takes time to run.

mh_neigh_data.tail():

          Borough    Neighborhood    Latitude    Longitude

      35    Manhattan    Turtle Bay    40.752042    -73.967708

      36    Manhattan    Tudor City    40.746917    -73.971219

      37    Manhattan    Stuyvesant Town    40.731000    -73.974052

      38    Manhattan    Flatiron    40.739673    -73.990947

      39    Manhattan    Hudson Yards    40.756658    -74.000111


Data 1 : Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

This dataset exists for free on the web. Link to the dataset is : https://geo.nyu.edu/catalog/nyu_2451_34572

Data 2 : Second data which will be used is the DOHMH Farmers Markets and Food Boxes dataset. In this we will be using the data of Farmers Markets.

https://data.cityofnewyork.us/dataset/DOHMH-Farmers-Markets-and-Food-Boxes/8vwk-6iz2

Website-https://www.grownyc.org/greenmarketco/foodbox GrowNYC's Fresh Food Box Program is a food access initiative that enables under-served communities to purchase fresh, healthy, and primarily regionally grown produce well below traditional retail prices.

A farmers' market is often defined as a public site used by two or more local or regional producers for the direct sale of farm products to consumers. In addition to fresh fruits and vegetables, markets may sell dairy products, fish, meat, baked goods, and other minimally processed foods.

Data 3 : For the below analysis we will get data from wikipedia as given below :

New York Population New York City Demographics Cuisine of New York city https://en.wikipedia.org/wiki/New_York_City https://en.wikipedia.org/wiki/Economy_of_New_York_City https://en.wikipedia.org/wiki/Portal:New_York_City https://en.wikipedia.org/wiki/Cuisine_of_New_York_City https://en.wikipedia.org/wiki/List_of_Michelin_starred_restaurants_in_New_York_City

Data 4 : Newyork city geographical coordinates data will be utilized as input for the Foursquare API, that will be leveraged to provision venues information for each neighborhood.We will use the Foursquare API to explore neighborhoods in New York City. The below is image of the Foursquare API data.

In [3]:
CLIENT_ID = 'VW4IYSJWZOV5MEUTYKEWPEINY3TZWYE0GJHVKNUQUSJ5DVFW'
CLIENT_SECRET = 'NORPTKWY0S0MX3LH53PPQPJ410DHEFFJMVU055BAKPE1QAH4'
VERSION = '20180604'
LIMIT = 30
print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: VW4IYSJWZOV5MEUTYKEWPEINY3TZWYE0GJHVKNUQUSJ5DVFW
CLIENT_SECRET:NORPTKWY0S0MX3LH53PPQPJ410DHEFFJMVU055BAKPE1QAH4


In [4]:
address = '222 W 23rd St, New York, NY 10011'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7443742 -73.9968175


In [5]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [6]:

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VW4IYSJWZOV5MEUTYKEWPEINY3TZWYE0GJHVKNUQUSJ5DVFW&client_secret=NORPTKWY0S0MX3LH53PPQPJ410DHEFFJMVU055BAKPE1QAH4&ll=40.7443742,-73.9968175&v=20180604&query=Italian&radius=500&limit=30'

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9d6ed714a126001b76e926'},
 'response': {'venues': [{'id': '5c893ef0d8096e0039e7363b',
    'name': 'Pisillo Italian Panini',
    'location': {'address': '124 W 25th St',
     'crossStreet': 'btwn 6th & 7th Ave',
     'lat': 40.744566,
     'lng': -73.992868,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.744566,
       'lng': -73.992868},
      {'label': 'entrance', 'lat': 40.744568, 'lng': -73.992984}],
     'distance': 333,
     'postalCode': '10001',
     'cc': 'US',
     'neighborhood': 'Chelsea',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['124 W 25th St (btwn 6th & 7th Ave)',
      'New York, NY 10001',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1c5941735',
      'name': 'Sandwich Place',
      'pluralName': 'Sandwich Places',
      'shortName': 'Sandwiches',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_',
       's

In [8]:
venues = results['response']['venues']
df = json_normalize(venues)
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,5c893ef0d8096e0039e7363b,Pisillo Italian Panini,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",v-1587375899,False,124 W 25th St,btwn 6th & 7th Ave,40.744566,-73.992868,"[{'label': 'display', 'lat': 40.744566, 'lng':...",333,10001,US,Chelsea,New York,NY,United States,"[124 W 25th St (btwn 6th & 7th Ave), New York,...",1108715,https://www.seamless.com/menu/pisillo-124-w-25...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
1,4a85a9e7f964a520d3fe1fe3,Italian Ice Cart,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1587375899,False,W 18th St,8th Ave,40.744268,-73.998099,"[{'label': 'display', 'lat': 40.74426816575664...",108,10011,US,NaN,New York,NY,United States,"[W 18th St (8th Ave), New York, NY 10011, Unit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59cbe33b50a6f0209c7b52c4,Claudio Italian Restaurant,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1587375899,False,344 8th Ave,26th & 27th Sts,40.747358,-73.996907,"[{'label': 'display', 'lat': 40.747358, 'lng':...",332,10001,US,NaN,New York,NY,United States,"[344 8th Ave (26th & 27th Sts), New York, NY ...",1229971,https://www.seamless.com/menu/claudios-pizzeri...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
3,5014183ce4b0073bd04161fa,Old Fashioned Hand Made Italian Ice,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1587375899,False,NaN,NaN,40.743568,-73.991700,"[{'label': 'display', 'lat': 40.74356804811844...",440,10023,US,NaN,New York,NY,United States,"[New York, NY 10023, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,526324c111d238e68dea4efd,Tre Stelle Italian Restaurant&Bar,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1587375899,False,39 west 24 th street,tra la 5th e la 6th avenue,40.743427,-73.991045,"[{'label': 'display', 'lat': 40.74342678284979...",498,10010,US,NaN,New York,NY,United States,[39 west 24 th street (tra la 5th e la 6th ave...,NaN,NaN,NaN,NaN,NaN,NaN,69667828


In [9]:
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
filtered_df = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
filtered_df['categories'] = filtered_df.apply(get_category_type, axis=1)

# clean column names by keeping only last term
filtered_df.columns = [column.split('.')[-1] for column in filtered_df.columns]

filtered_df

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Pisillo Italian Panini,Sandwich Place,124 W 25th St,btwn 6th & 7th Ave,40.744566,-73.992868,"[{'label': 'display', 'lat': 40.744566, 'lng':...",333,10001,US,Chelsea,New York,NY,United States,"[124 W 25th St (btwn 6th & 7th Ave), New York,...",5c893ef0d8096e0039e7363b
1,Italian Ice Cart,Food Truck,W 18th St,8th Ave,40.744268,-73.998099,"[{'label': 'display', 'lat': 40.74426816575664...",108,10011,US,NaN,New York,NY,United States,"[W 18th St (8th Ave), New York, NY 10011, Unit...",4a85a9e7f964a520d3fe1fe3
2,Claudio Italian Restaurant,Pizza Place,344 8th Ave,26th & 27th Sts,40.747358,-73.996907,"[{'label': 'display', 'lat': 40.747358, 'lng':...",332,10001,US,NaN,New York,NY,United States,"[344 8th Ave (26th & 27th Sts), New York, NY ...",59cbe33b50a6f0209c7b52c4
3,Old Fashioned Hand Made Italian Ice,Food Truck,NaN,NaN,40.743568,-73.991700,"[{'label': 'display', 'lat': 40.74356804811844...",440,10023,US,NaN,New York,NY,United States,"[New York, NY 10023, United States]",5014183ce4b0073bd04161fa
4,Tre Stelle Italian Restaurant&Bar,Italian Restaurant,39 west 24 th street,tra la 5th e la 6th avenue,40.743427,-73.991045,"[{'label': 'display', 'lat': 40.74342678284979...",498,10010,US,NaN,New York,NY,United States,[39 west 24 th street (tra la 5th e la 6th ave...,526324c111d238e68dea4efd
5,Maffel Italian Eatery,Pizza Place,NaN,NaN,40.741749,-73.992574,"[{'label': 'display', 'lat': 40.741749, 'lng':...",462,10010,US,NaN,New York,NY,United States,"[New York, NY 10010, United States]",50393bd2e4b0014516101a0e
6,La Cucina Italiana,Italian Restaurant,512 Fashion Ave,W 38th St,40.744667,-74.000964,"[{'label': 'display', 'lat': 40.744667, 'lng':...",351,10018,US,NaN,New York,NY,United States,"[512 Fashion Ave (W 38th St), New York, NY 100...",4b0af88cf964a520e22a23e3
7,biricchino,Italian Restaurant,260 W 29th St,at 8th Ave.,40.748874,-73.995469,"[{'label': 'display', 'lat': 40.74887439608574...",513,10001,US,NaN,New York,NY,United States,"[260 W 29th St (at 8th Ave.), New York, NY 100...",4af4b9adf964a52074f521e3
8,Salumeria Biellese Delicatessen,Deli / Bodega,378 8th Ave,at W 29th St,40.749081,-73.995606,"[{'label': 'display', 'lat': 40.74908132149613...",533,10001,US,NaN,New York,NY,United States,"[378 8th Ave (at W 29th St), New York, NY 1000...",4a71be2bf964a52070d91fe3


In [10]:

filtered_df.name

0                 Pisillo Italian Panini
1                       Italian Ice Cart
2             Claudio Italian Restaurant
3    Old Fashioned Hand Made Italian Ice
4      Tre Stelle Italian Restaurant&Bar
5                  Maffel Italian Eatery
6                     La Cucina Italiana
7                             biricchino
8        Salumeria Biellese Delicatessen
Name: name, dtype: object

The fifth cluster has one neighborhood which consists of Venues such as Grocery shops, Bars, Restaurants, Furniture shops, and Department stores.

Results and Discussion 
The aim of this project is to help people who want to relocate to the safest borough in London, expats can chose the neighborhoods to which they want to relocate based on the most common venues in it. For example if a person is looking for a neighborhood with good connectivity and public transportation we can see that Clusters 3 and 4 have Train stations and Bus stops as the most common venues. If a person is looking for a neighborhood with stores and restaurants in a close proximity then the neighborhoods in the first cluster is suitable. For a family I feel that the neighborhoods in Cluster 4 are more suitable dues to the common venues in that cluster, these neighborhoods have common venues such as Parks, Gym/Fitness centers, Bus Stops, Restaurants, Electronics Stores and Soccer fields which is ideal for a family.

Conclusion 
This project helps a person get a better understanding of the neighborhoods with respect to the most common venues in that neighborhood. It is always helpful to make use of technology to stay one step ahead i.e. finding out more about places before moving into a neighborhood. We have just taken safety as a primary concern to shortlist the borough of London. The future of this project includes taking other factors such as cost of living in the areas into consideration to shortlist the borough based on safety and a predefined budget.